## All models compared to each other

In [20]:
import pandas as pd

data = pd.read_csv('data/StudentPerformanceFactors.csv')

X = data.drop("Exam_Score", axis=1)
Y = data['Exam_Score']

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.75)

num_cols = X.select_dtypes(include=['int64', 'float64']).columns
cat_cols = X.select_dtypes(include=['object', 'category']).columns

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

from sklearn.preprocessing import OneHotEncoder

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols)
])

### Models
#### Linear Regression

In [21]:
from sklearn.linear_model import LinearRegression

sklearn_LR_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

sklearn_LR_pipeline.fit(X_train, Y_train)

y_pred_lr_skl = sklearn_LR_pipeline.predict(X_test)

names = ['Linear Regression (sklearn)']
y_predictions = [y_pred_lr_skl]

#### Linear Regression less features (12)

In [86]:
from sklearn.feature_selection import SelectKBest, f_regression

small_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('selector', SelectKBest(score_func=f_regression, k=12)),
    ('regressor', LinearRegression())
])

small_pipeline.fit(X_train, Y_train)

y_pred_small = small_pipeline.predict(X_test)

names.append('Linear Regression SelectKBest')
y_predictions.append(y_pred_small)

#### Linear Regression Polynomial Features

In [87]:
from sklearn.preprocessing import PolynomialFeatures

poly_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures(degree=2)),
    ('regressor', LinearRegression())
])

poly_pipeline.fit(X_train, Y_train)

y_pred_poly = poly_pipeline.predict(X_test)

names.append('Polynomial Features')
y_predictions.append(y_pred_poly)

#### Lasso

In [88]:
from sklearn.linear_model import Lasso

L1_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Lasso())
])

from sklearn.model_selection import GridSearchCV
import numpy as np

L1_param_grid = {
    'regressor__alpha': np.logspace(-5, -1, 9)
}

l1_grid = GridSearchCV(L1_pipeline, L1_param_grid, cv=5, scoring='r2', n_jobs=-1)
l1_grid.fit(X_train, Y_train)

print("Lasso Best Params:", l1_grid.best_params_)

y_pred_l1 = l1_grid.predict(X_test)

names.append('Lasso')
y_predictions.append(y_pred_l1)

Lasso Best Params: {'regressor__alpha': np.float64(0.0031622776601683794)}


#### Ridge

In [89]:
from sklearn.linear_model import Ridge

L2_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

L2_param_grid = {
    'regressor__alpha': np.linspace(1, 10, 10)
}

l2_grid = GridSearchCV(L2_pipeline, L2_param_grid, cv=5, scoring='r2', n_jobs=-1)
l2_grid.fit(X_train, Y_train)

print("Ridge Best Params:", l2_grid.best_params_)

y_pred_l2 = l2_grid.predict(X_test)

names.append('Ridge')
y_predictions.append(y_pred_l2)

Ridge Best Params: {'regressor__alpha': np.float64(10.0)}


#### Linear Regression closed form solution

In [22]:
from src.linear_regression.models import LinearRegressionClosedForm

closed_form_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegressionClosedForm())
])

closed_form_pipeline.fit(X_train, Y_train)

y_pred_lr_cf = closed_form_pipeline.predict(X_test)

names.append('Closed Form Solution')
y_predictions.append(y_pred_lr_cf)

#### Linear Regression closed form solution WITH L2 regularization

In [23]:
cf_LR_L2_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegressionClosedForm(regularization=True, alpha=10))
])

cf_LR_L2_pipeline.fit(X_train, Y_train)

y_pred_cf_l2 = cf_LR_L2_pipeline.predict(X_test)

names.append('Closed Form Solution with L2')
y_predictions.append(y_pred_cf_l2)

#### Linear Regression gradient descent

In [24]:
from src.linear_regression.models import LinearRegressionGradientDescent

gradient_descent_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegressionGradientDescent())
])

gradient_descent_pipeline.fit(X_train, Y_train)

y_pred_lr_gd = gradient_descent_pipeline.predict(X_test)

names.append('Gradient Descent')
y_predictions.append(y_pred_lr_gd)

#### Linear Regression gradient descent WITH L1 regularization

In [25]:
gd_l1_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegressionGradientDescent(regularization='l1', alpha=5))
])

gd_l1_pipeline.fit(X_train, Y_train)

y_pred_gd_l1 = gd_l1_pipeline.predict(X_test)

names.append('Gradient Descent L1')
y_predictions.append(y_pred_gd_l1)

#### Linear Regression gradient descent WITH L2 regularization

In [26]:
gd_l2_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegressionGradientDescent(regularization='l2', alpha=0.5))
])

gd_l2_pipeline.fit(X_train, Y_train)

y_pred_gd_l2 = gd_l2_pipeline.predict(X_test)

names.append('Gradient Descent L2')
y_predictions.append(y_pred_gd_l2)

#### Linear Regression gradient descent WITH both L1 and L2

In [27]:
gd_en_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegressionGradientDescent(regularization='elasticnet', alpha=1))
])

gd_en_pipeline.fit(X_train, Y_train)

y_pred_gd_en = gd_en_pipeline.predict(X_test)

names.append('Gradient Descent L1 and L2')
y_predictions.append(y_pred_gd_en)

#### Support Vector Regression (grid search)

In [96]:
from sklearn.svm import SVR

SVR_grid_search = SVR()

svr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR_grid_search)
])

svr_param_grid = {
    'regressor__C': [0.1, 1, 5, 10, 20],
    'regressor__epsilon': [0.05, 0.1, 0.5],
    'regressor__gamma': ['scale', 0.01, 0.1]
}

svr_grid = GridSearchCV(svr_pipeline, svr_param_grid, cv=5, scoring='r2', n_jobs=-1)
svr_grid.fit(X_train, Y_train)

y_pred_svr_grid = svr_grid.predict(X_test)

In [97]:
print(svr_grid.best_params_)

names.append('Support Vector Regression')
y_predictions.append(y_pred_svr_grid)

{'regressor__C': 5, 'regressor__epsilon': 0.5, 'regressor__gamma': 0.01}


#### Random Forest Regression (grid search)

In [98]:
from sklearn.ensemble import RandomForestRegressor

RFR_grid_search = RandomForestRegressor(random_state=42)

rfr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RFR_grid_search)
])

rf_param_grid = {
    'regressor__n_estimators': [100, 200, 300],
    'regressor__max_depth': [15, 20, 25],
    'regressor__min_samples_split': [7, 10, 18],
    'regressor__max_features': ['sqrt', 0.5, 1]
}

rf_grid = GridSearchCV(rfr_pipeline, rf_param_grid, cv=5, scoring='r2', n_jobs=-1)
rf_grid.fit(X_train, Y_train)

y_pred_rf_grid = rf_grid.predict(X_test)

In [99]:
names.append('Random Forest Regression')
y_predictions.append(y_pred_rf_grid)

print(rf_grid.best_params_)

{'regressor__max_depth': 20, 'regressor__max_features': 0.5, 'regressor__min_samples_split': 7, 'regressor__n_estimators': 300}


### Evaluation

In [28]:
from src.linear_regression.model_evaluation import metrics_table

df_metrics = metrics_table(Y_test, y_predictions, names)
df_sorted = df_metrics.sort_values(by='R²', ascending=False)

display(df_sorted)

,R²,MSE,RMSE,MAE
Closed Form Solution with L2,0.716410,4.192945,2.047668,0.491163
Linear Regression (sklearn),0.716281,4.194855,2.048135,0.491390
Closed Form Solution,0.716281,4.194855,2.048135,0.491390
Gradient Descent,0.715468,4.206868,2.051065,0.531695
Gradient Descent L2,0.593540,6.009604,2.451449,1.319385
Gradient Descent L1 and L2,0.480461,7.681506,2.771553,1.715653
Gradient Descent L1,0.005269,14.707329,3.835014,2.791925


#### Cross-validation

In [29]:
from sklearn.model_selection import cross_validate, KFold

kfold = KFold(n_splits=3, shuffle=True, random_state=42)

estimators = [sklearn_LR_pipeline,
              #small_pipeline,
              #poly_pipeline,
              #l1_grid.best_estimator_,
              #l2_grid.best_estimator_,
              closed_form_pipeline,
              cf_LR_L2_pipeline,
              gradient_descent_pipeline,
              gd_l1_pipeline,
              gd_l2_pipeline,
              gd_en_pipeline,
              #svr_grid.best_estimator_,
              #rf_grid.best_estimator_
              ]

cross_vals = []

for estimator in estimators:
    cross_val = cross_validate(estimator, X_train, Y_train, cv=kfold,
                              scoring=('r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'),
                              return_train_score=True)
    cross_vals.append(cross_val)

In [30]:
import numpy as np

metrics = ['R² (test)', 'R² (train)', 'R² difference', 'MSE (test)', 'MAE (test)']

results = {
    name: [
        np.mean(cross_val['test_r2']),
        np.mean(cross_val['train_r2']),
        np.mean(cross_val['train_r2']) - np.mean(cross_val['test_r2']),
        -np.mean(cross_val['test_neg_mean_squared_error']),
        -np.mean(cross_val['test_neg_mean_absolute_error'])
    ]
    for (name, cross_val) in zip(names, cross_vals) 
}

df_cross_val = pd.DataFrame(results, index=metrics).T.round(6)
df_sorted_cross_val = df_cross_val.sort_values(by='R² (test)', ascending=False)

display(df_sorted_cross_val)

,R² (test),R² (train),R² difference,MSE (test),MAE (test)
Gradient Descent,0.728449,0.732094,0.003645,4.190231,0.509112
Closed Form Solution with L2,0.728418,0.732142,0.003724,4.190484,0.505465
Linear Regression (sklearn),0.728405,0.732171,0.003766,4.190884,0.506094
Closed Form Solution,0.728405,0.732171,0.003766,4.190884,0.506094
Gradient Descent L2,0.599851,0.602256,0.002405,6.132705,1.336414
Gradient Descent L1 and L2,0.484754,0.485526,0.000771,7.878257,1.732111
Gradient Descent L1,0.007844,0.007388,-0.000456,15.127139,2.841914


In [19]:
print('Intercept before regularization:')
print('\t', closed_form_pipeline['regressor'].intercept_)
print('Intercept after regularization:')
print('\t', cf_LR_L2_pipeline['regressor'].intercept_)

Intercept before regularization:
	 10.72954692975006
Intercept after regularization:
	 66.17996464135784
